<a href="https://colab.research.google.com/github/guiaech/Deteccao_humana_com_visao_computacional/blob/main/Deteccao_humana_com_visao_computacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Python - Detecção e contagem humana em tempo real**
Neste projeto python, iremos construir o Sistema de Detecção e Contagem Humana através da Webcam ou você pode fornecer seu próprio vídeo ou imagens. Este é um projeto de aprendizado profundo.


# **Pré-requisitos do projeto**

**OpenCV:**  uma biblioteca forte usada para aprendizado de máquina

**Imutils:** processamento da imagem

**Numpy:**  usado para computação científica. A imagem é armazenada em uma matriz numpy.

**Argparse:**  usado para fornecer entrada na linha de comando.

In [1]:
#INSTALANDO AS BIBLIOTECAS NECESSÁRIAS
!pip install opencv-python
!pip install imutils
!pip install numpy

# **Histograma do Descritor de Gradiente Orientado**
HOG é um descritor de recurso usado em visão computacional e processamento de imagem para fins de detecção de objetos. Esta é uma das técnicas mais populares para detecção de objetos, para nossa sorte, o OpenCV já foi implementado de forma eficiente para combinar o algoritmo HOG Descriptor com Support Vector Machine ou SVM.

In [3]:
#IMPORTANDO AS BIBLIOTECAS
import cv2
import imutils
import numpy as np
import argparse

# **Criando um modelo que detectará Humanos:**

Conforme discutido anteriormente, usaremos HOGDescriptor com SVM já implementado no OpenCV. O código abaixo fará este trabalho:

**cv2.HOGDescriptor_getDefaultPeopleDetector()** chama o modelo pré-treinado para detecção humana de OpenCV e então alimentaremos nossa máquina de vetores de suporte com ele.

In [4]:
HOGCV = cv2.HOGDescriptor()
HOGCV.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# **Detect() method:**

Aqui, a mágica real acontecerá.

**Vídeo:** um vídeo combina uma sequência de imagens para formar uma imagem em movimento. Chamamos essas imagens de Frame. Portanto, em geral, detectaremos a pessoa no quadro. E mostre um após o outro que parece um vídeo.

Isso é exatamente o que nosso método Detect () fará. Será necessário um quadro para detectar uma pessoa nela. Faça uma caixa ao redor de uma pessoa e mostre a moldura ... e devolva a moldura com a pessoa delimitada por uma caixa verde.


Tudo será feito por detectMultiScale(). Ele retorna 2 tuplas.

Lista contendo coordenadas da caixa delimitadora de pessoa.
As coordenadas estão na forma X, Y, W, H.
Onde x, y são as coordenadas iniciais da caixa ew, h são a largura e a altura da caixa, respectivamente.
Valor de confiança de que é uma pessoa.

In [5]:
def detect(frame):
    bounding_box_cordinates, weights =  HOGCV.detectMultiScale(frame, winStride = (4, 4), padding = (8, 8), scale = 1.03)
    
    person = 1
    for x,y,w,h in bounding_box_cordinates:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(frame, f'person {person}', (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
        person += 1
    
    cv2.putText(frame, 'Status : Detecting ', (40,40), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.putText(frame, f'Total Persons : {person-1}', (40,70), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.imshow('output', frame)

    return frame

# **Método HumanDetector ()**

Existem duas maneiras de obter vídeo.

Câmera web

Caminho do arquivo armazenado
Neste projeto de aprendizado profundo, podemos tirar fotos também. Portanto, nosso método irá verificar se um caminho é fornecido e, em seguida, procurar o vídeo ou imagem no caminho fornecido e operar. Caso contrário, a webCam será aberta.

In [7]:
def humanDetector(args):
    image_path = args["image"]
    video_path = args['video']
    if str(args["camera"]) == 'true' : camera = True 
    else : camera = False

    writer = None
    if args['output'] is not None and image_path is None:
        writer = cv2.VideoWriter(args['output'],cv2.VideoWriter_fourcc(*'MJPG'), 10, (600,600))

    if camera:
        print('[INFO] Opening Web Cam.')
        detectByCamera(ouput_path,writer)
    elif video_path is not None:
        print('[INFO] Opening Video from path.')
        detectByPathVideo(video_path, writer)
    elif image_path is not None:
        print('[INFO] Opening Image from path.')
        detectByPathImage(image_path, args['output'])

# **Método DetectByCamera ()**

cv2.VideoCapture (0) passando 0 nesta função significa que queremos gravar de uma webcam. video.read () lê quadro a quadro. Ele retorna uma marca de verificação que é Verdadeira se isso foi capaz de ler um quadro de outra forma False.
Agora, para cada quadro, chamaremos o método detect (). Em seguida, escrevemos o quadro em nosso arquivo de saída.

In [8]:
def detectByCamera(writer):   
    video = cv2.VideoCapture(0)
    print('Detecting people...')

    while True:
        check, frame = video.read()

        frame = detect(frame)
        if writer is not None:
            writer.write(frame)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()